<a href="https://colab.research.google.com/github/sartajsehgal/Data-Science/blob/main/Kinship_Relationship_DSEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model
import pandas as pd
from tqdm import tqdm

##**Loading the train and test Data**
We are going to make predictions using the pre-trained facenet model on the SMILE dataset which is given on the KAGGLE competition. The link to the competition is: https://www.kaggle.com/c/recognizing-faces-in-the-wild. The link of the pretrained model is: https://github.com/nyoki-mtl/keras-facenet.

The first task is to import the train_relations file and the sample submission file to make the train dataframe and the test dataframe. The train_relationships.csv contains the two people p1 and p2 which are related to each other.

In [2]:
train_df = pd.read_csv("/content/drive/MyDrive/Kaggle/SMILE_dataset/train_relationships.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Kaggle/SMILE_dataset/sample_submission.csv")

##**Importing the Model**
I have downloaded the facenet_keras model to the google drive from To join the video meeting, click this link: https://github.com/nyoki-mtl/keras-facenet. The FaceNet model takes as input the image of a person’s face, produces a vector embedding of 128 numbers, which are then projected in a high-dimensional Euclidean space. Here, the distance between points corresponds to a measure of face similarity.

In [3]:
model_path = "/content/drive/MyDrive/Kaggle/SMILE_dataset/keras-facenet/keras-facenet/model/facenet_keras.h5"
model = load_model(model_path)

##**Data Preprocessing**
**Prewhitening** - Prewhitening is the process of eliminating or reducing 
short-term stochastic persistence to enable detection of deterministic change.
It subtracts the average and normalizes the range of the pixel values of input images.

**Normalization** - We normalize all the values.

**Load and Align Images** - In the end we load the images from the specified path and align it.


In [4]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def load_and_align_images(filepaths, margin,image_size = 160):
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)
        aligned = resize(img, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

##**Calculating Embeddings**
Now, we calculate the Embeddings of the test images using the pretrained Facenet Model.

In [5]:
def calc_embs(filepaths, margin=10, batch_size=512):
    pd = []
    for start in tqdm(range(0, len(filepaths), batch_size)):
        aligned_images = prewhiten(load_and_align_images(filepaths[start:start+batch_size], margin))
        pd.append(model.predict_on_batch(aligned_images))
    embs = l2_normalize(np.concatenate(pd))

    return embs

Importing the test Images and then calling the calc_embs function for calculating the embeddings of all the test images.

In [6]:
test_images = os.listdir("/content/drive/MyDrive/Kaggle/SMILE_dataset/test/")
test_embs = calc_embs([os.path.join("/content/drive/MyDrive/Kaggle/SMILE_dataset/test/", f) for f in test_images])
test_embs

100%|██████████| 13/13 [06:28<00:00, 29.91s/it]


array([[ 0.03484537,  0.12908678, -0.0181959 , ..., -0.2585555 ,
        -0.13148756, -0.01973889],
       [ 0.04109402,  0.04779609,  0.12647359, ..., -0.07085994,
         0.05687727,  0.044689  ],
       [-0.0494254 , -0.01584665,  0.08263745, ..., -0.05969948,
         0.10659206, -0.14784393],
       ...,
       [ 0.06496517,  0.02294946,  0.07013849, ..., -0.2266992 ,
        -0.05978282, -0.01152707],
       [ 0.05065063, -0.05268567, -0.08779337, ...,  0.04090406,
         0.03944483, -0.00751425],
       [-0.04110827,  0.01781067, -0.0139101 , ..., -0.03060862,
        -0.11479206,  0.02817542]], dtype=float32)

In [7]:
test_embs.shape

(6282, 128)

In [8]:
test_embs[0].shape

(128,)

Introducing a new column for calculating distance in the test dataframe and assigning it value zero.
Making a dictionary for tracking the index and name of every image

In [9]:
test_df["distance"] = 0
img2idx = dict()
for idx, img in enumerate(test_images):
    img2idx[img] = idx

In [10]:
img2idx

{'face00111.jpg': 0,
 'face04253.jpg': 1,
 'face05610.jpg': 2,
 'face05478.jpg': 3,
 'face05298.jpg': 4,
 'face02061.jpg': 5,
 'face03115.jpg': 6,
 'face04708.jpg': 7,
 'face00826.jpg': 8,
 'face03021.jpg': 9,
 'face04999.jpg': 10,
 'face00696.jpg': 11,
 'face01787.jpg': 12,
 'face00552.jpg': 13,
 'face04364.jpg': 14,
 'face03354.jpg': 15,
 'face01039.jpg': 16,
 'face01392.jpg': 17,
 'face01252.jpg': 18,
 'face04815.jpg': 19,
 'face03253.jpg': 20,
 'face00787.jpg': 21,
 'face01640.jpg': 22,
 'face02465.jpg': 23,
 'face00106.jpg': 24,
 'face00449.jpg': 25,
 'face02631.jpg': 26,
 'face05683.jpg': 27,
 'face04068.jpg': 28,
 'face01473.jpg': 29,
 'face00066.jpg': 30,
 'face03394.jpg': 31,
 'face01444.jpg': 32,
 'face02584.jpg': 33,
 'face02804.jpg': 34,
 'face05745.jpg': 35,
 'face06184.jpg': 36,
 'face00115.jpg': 37,
 'face00138.jpg': 38,
 'face02007.jpg': 39,
 'face06031.jpg': 40,
 'face01523.jpg': 41,
 'face05747.jpg': 42,
 'face04496.jpg': 43,
 'face01138.jpg': 44,
 'face03199.jpg': 45

##**Calculating distance between 2 images**
Now, we are going to calculate the distance between the embeddings of the images of two people. We know that the distance will determine the similarity and difference between images of two people which will help us determine if the two people belong to the same family or not. This is because there must be some kind of similarity in faces of people of the same family, e.g. similar shape of nose, lips, eyes etc. For calculating distance we first split the names and then calculate embedding for each image.

In [11]:
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    imgs = [test_embs[img2idx[img]] for img in row.img_pair.split("-")]
    test_df.loc[idx, "distance"] = distance.euclidean(*imgs)

100%|██████████| 5310/5310 [00:02<00:00, 2539.77it/s]


Adding the values of distances for all the relations given for test.

In [12]:
all_distances = test_df.distance.values
sum_dist = np.sum(all_distances)

##**Calculating Probabilities**
Now we are calculating the cummulative probability of whether the two people belong to the same family or not. 
We are subtracting the final probabilities from 1 because the more is the distance the lesser is the chance of two people belonging to the same family.

In [13]:
probs = []
for dist in tqdm(all_distances):
    prob = np.sum(all_distances[np.where(all_distances <= dist)[0]])/sum_dist
    probs.append(1 - prob)

100%|██████████| 5310/5310 [00:00<00:00, 23582.14it/s]


Generating the final csv file for submission.

In [14]:
sub_df = pd.read_csv("/content/drive/MyDrive/Kaggle/SMILE_dataset/sample_submission.csv")
sub_df.is_related = probs
sub_df.to_csv("submission.csv", index=False)